In [8]:
import pandas as pd
import time
import string
import networkx as nx
from networkx import Graph
# from pyvis.network import Network
import ast
import random
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import hvplot.networkx as hvnx
import ipywidgets as widgets
from IPython.display import display, clear_output, IFrame
# import numpy as np
from tabulate import tabulate

In [9]:
df_concat_list = []

for i in range(30):
    df_number = i+1
    edge_df = pd.read_csv(f'test_edges{df_number}.csv', delimiter='\t', low_memory=False, encoding = 'utf-8', dtype = {'Source': str, 'Target': str})
    df_concat_list.append(edge_df)
    # print(len(df_concat_list))

edges = pd.concat(df_concat_list, axis=0, ignore_index = True)

In [10]:
#%%time
# calculating node properties
graph = nx.from_pandas_edgelist(edges, "Source", "Target", edge_attr = ['title', 'link'], create_using=Graph)

In [11]:
#%%time
#Computing centrality
deg_cent = nx.degree_centrality(graph)
#Descending order sorting centrality
deg_cent_sorted= dict(sorted(deg_cent.items(), key=lambda item: item[1],reverse=True))
deg_cent_sorted_list = sorted(deg_cent.items(), key=lambda item: item[1],reverse=True)

In [12]:
#%%time
graph_nodes = list(graph.nodes)

In [13]:
# Gets all the neighbors of a node and their identified relationships
def g_ego(graph, node = "trust", rad = 1):
    Ego = nx.ego_graph(graph, node, radius=rad, center=True, undirected=True)
    return Ego

def minimum(a, b):
    if a <= b:
        return a
    else:
        return b

In [14]:
#%%time

search_term_1 = widgets.Text(placeholder='Variable/Construct')
search_param_1 = widgets.ToggleButtons(options=[1, 2],
                                       #    value='pineapple', # Defaults to 'pineapple'
                                       #    layout={'width': 'max-content'}, # If the items' names are long
                                       description='Radius',
                                       disabled=False,
                                       button_style=''
                                      )


search_param_2 = widgets.Checkbox(value=False,
                                  description='List All Inclusive Terms',
                                  disabled=False,
                                  indent=False
                                 )

search_param_3 = widgets.Checkbox(value=False,
                                  description='List Influential Related Nodes',
                                  disabled=False,
                                  indent=False
                                 )


search_param_4 = widgets.Checkbox(value=False,
                                  description='Article Recommendation',
                                  disabled=False,
                                  indent=False
                                 )

display_param_1 = widgets.Checkbox(value=False,
                                   description='Display (for single ego networks with 1 radius)',
                                   disabled=False,
                                   indent=False
                                  )




button_search = widgets.Button(description='Look into it',tooltip='Send',style={'description_width': 'initial'})

output = widgets.Output()


def on_search_clicked(event):
    with output:
        clear_output()
        
        if search_param_2.value == False:
            rtest1 = g_ego(graph, node=search_term_1.value, rad = int(search_param_1.value)) 
            
        if search_param_2.value == True:
            rtest1 = g_ego(graph, node=search_term_1.value, rad = int(search_param_1.value)) 
            search_list = []
            
            for node in graph_nodes:
                if search_term_1.value in str(node):
                    search_list.append(str(node))
            print(f'Inclusive Nodes:\nThe number of inclusive nodes: {len(search_list)}\n{search_list}\n\n\n')

            graph_list = []            
            for term in search_list:
                graph_list.append(g_ego(graph, node=term, rad = int(search_param_1.value)))
                
            rtest1 = graph_list[0]
            for g in range(len(graph_list)-1): 
                rtest1 = nx.compose(rtest1, graph_list[g+1])

        node_rec = 30
        article_rec = 30
        
        # 'List Influential Related Nodes'
        if search_param_3.value == True:
            search_list = []
            for node in graph_nodes:
                if search_term_1.value in str(node):
                    search_list.append(str(node))
            
            construct_list = []
            node_rec_list = []
            sub_nodes = list(rtest1.nodes())
            counter = minimum(len(sub_nodes), node_rec)
            for i in range(len(deg_cent_sorted_list)):
                for j in sub_nodes:
                    if j == deg_cent_sorted_list[i][0] and len(node_rec_list)<counter and j not in construct_list:
                        node_rec_list.append(j)
            
            print_string = ''

            if node_rec_list[-1] not in search_list:
                for c in range(counter-1):
                    if  str(node_rec_list[c]) not in search_list:
                        print_string = print_string+node_rec_list[c]+', '
                
                print_string = print_string+node_rec_list[-1]
                
                print(f"Influential Related Nodes:\n{print_string}\n\n\n")
            if node_rec_list[-1] in search_list:
                for c in range(counter-2):
                    if  str(node_rec_list[c]) not in search_list:
                        print_string = print_string+node_rec_list[c]+', '
                
                print_string = print_string+node_rec_list[-2]
                
                print(f"Influential Related Nodes:\n{print_string}\n\n\n")
        
            #print(node_rec_list[:counter])
            
        # 'Article Recommendation'
        if search_param_4.value == True:
            link_list = []
            node_rec_list = []
            article_rec_list = []
            rec_source_list = []
            rec_target_list = []
            
            sub_nodes = list(rtest1.nodes())
            
            counter = minimum(len(sub_nodes), article_rec)
            for i in range(len(deg_cent_sorted_list)):
                for j in sub_nodes:
                    if j == deg_cent_sorted_list[i][0] and len(article_rec_list)<counter:
                        node_rec_list.append(j)

            
            
            rec_edges = list(rtest1.edges(data=True))#list(rtest1.edges(search_term_1.value, data=True))
            
            for k in node_rec_list:
                for l in range(len(rec_edges)):
                    if rec_edges[l][0] == k or rec_edges[l][1] == k:
                        if len(article_rec_list)<=counter and rec_edges[l][2]['link'] not in link_list:
                            if rec_edges[l][0] == search_term_1.value or rec_edges[l][1] == search_term_1.value:
                                link_list.append(rec_edges[l][2]['link'])
                                rec_source_list.append(rec_edges[l][0])
                                rec_target_list.append(rec_edges[l][1])
                                article_rec_list.append([rec_edges[l][0], rec_edges[l][1], rec_edges[l][2]['title'], rec_edges[l][2]['link']])

                            
                            node_rec_count = rec_source_list.count(rec_edges[l][0]) + rec_source_list.count(rec_edges[l][1]) + rec_target_list.count(rec_edges[l][0]) + rec_target_list.count(rec_edges[l][1])
                            search_list_2 = []
                            
                            for node in graph_nodes:
                                if search_term_1.value in str(node):
                                    search_list_2.append(str(node))
                            
                            
                            if rec_edges[l][0] in search_list_2 or rec_edges[l][1] in search_list_2:
                                if rec_edges[l][2]['link'] not in link_list:
                                    link_list.append(rec_edges[l][2]['link'])
                                    rec_source_list.append(rec_edges[l][0])
                                    rec_target_list.append(rec_edges[l][1])
                                    article_rec_list.append([rec_edges[l][0], rec_edges[l][1], rec_edges[l][2]['title'], rec_edges[l][2]['link']])


            print(tabulate(article_rec_list, headers=['Source', 'Target', 'Title', 'Link']))
            
            # print(article_rec_list[:counter])

        
        # display
        if display_param_1.value == True:
            dis_graph = g_ego(graph, node=search_term_1.value, rad = int(search_param_1.value))
            pos = nx.spring_layout(dis_graph)
            display(hvnx.draw(dis_graph, pos, width = 2000, height = 2000, node_size = 200, iterations = 60, arrows = True, arrowhead_length = 0.007))#, k = ((len(graph_nodes)/100))))#, with_labels = True))

        # export
        edge_list = list(rtest1.edges(data=True))
        graph_df = nx.to_pandas_edgelist(rtest1)
        
        button_export = widgets.Button(description='Export Network Data',tooltip='Send',style={'description_width': 'initial'})    
        button_export.on_click(graph_df.to_csv(f'ego_{search_term_1.value}_{search_param_1.value}_{search_param_2.value}.csv', sep='\t', encoding = 'utf-8', index=False))

        vbox_export = widgets.VBox([button_export])
        display(vbox_export)
        
button_search.on_click(on_search_clicked)

vbox_result = widgets.VBox([search_term_1, search_param_1, search_param_2, search_param_3, search_param_4, display_param_1, button_search, output])
display(vbox_result)

In [15]:
# !pip freeze > requirements.txt